##### Iain's comments
In some of the cases it would be good to use more informative variable names e.g. _res, _class, etc. I would change res to pred_score, pred_prob, or pred_logists or something like that.
The function roc_auc_score_multiclass() is missing index in the argument.
I’m pretty sure pred_class should be named pred_proba/pred_score etc, not pred_class (pred_class sounds like it is the predicted class, but this is actually a continuous prediction).
Double check if there is a problem with precision_score.recall_score/f1_score and the pos_label argument e.g. the positive label should be 1.
In roc_auc_score_multiclass(), I would make the elements of eval_dict a dict, not a list where you have to remember the order of the input (see attached script)
I’ve attached a script for  computing various multiclass classification metrics. It’s not finished, but it’s a bit more streamlined and will give us more information. For classification – especially mutlitclass problems – we want to compute a ton of metrics to help understand what’s going on

In [1]:
# importing the libraries
import os
import sys
import pandas as pd
import torch
from PIL import Image
import numpy as np
import pandas as pd
import cv2
import  glob
import time
import albumentations
import math
import shutil
from collections import Counter
from scipy.special import softmax
from sklearn.preprocessing import OneHotEncoder# creating instance of one-hot-encoder
from sklearn.model_selection import train_test_split

from models.ResNext50 import Myresnext50
from train.train_classification import trainer_classification
from utils.utils import configure_optimizers
from Datasets.DataLoader import Img_DataLoader

### PyTorch Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils import data
from torchvision import transforms


In [2]:
### ORIGINAL CODE BY DEEPHEME, REPLACED BY CELL BELOW BUT KEPT FOR REFERENCE
# X_train = glob.glob('../../2022_05_18_cells_50_NORMAL/Cross_Validation/iteration_3/train/*/*')
# X_val = glob.glob('../../2022_05_18_cells_50_NORMAL/Cross_Validation/iteration_3/val/*/*')

# labels = [x.split('/')[-2] for x in X_train]
# cell_types = set(labels)

# cell_types = list(cell_types)
# cell_types.sort()

# cell_types_df = pd.DataFrame(cell_types, columns=['Cell_Types'])# converting type of columns to 'category'
# cell_types_df['Cell_Types'] = cell_types_df['Cell_Types'].astype('category')# Assigning numerical values and storing in another column
# cell_types_df['Cell_Types_Cat'] = cell_types_df['Cell_Types'].cat.codes

# enc = OneHotEncoder(handle_unknown='ignore')# passing bridge-types-cat column (label encoded values of bridge_types)
# enc_df = pd.DataFrame(enc.fit_transform(cell_types_df[['Cell_Types_Cat']]).toarray())# merge with main df bridge_df on key values
# cell_types_df = cell_types_df.join(enc_df)

In [12]:
# Augmented code to store the data in two folders, uncomment if you want to store the files in a directory

# # Read data
# cell_types_df = pd.read_pickle("imagepaths.pkl")

# # First split into training and validation sets
# train_df, val_df = train_test_split(cell_types_df, test_size=0.15, random_state=42, stratify=cell_types_df['Label'])
# print(f"Training set shape: {train_df.shape}\nValidation set shape: {val_df.shape}")

# # Copy the images of the training and validation sets into a directory to use the same approach as Deepheme
# def copy_images(df_subset, target_dir):
#     for file_path in df_subset['Filepath']:
#         # Get just the filename (without directories)
#         file_name = os.path.basename(file_path)
#         # Define the destination path
#         dest_path = os.path.join(target_dir, file_name)
#         # Copy the file
#         shutil.copy(file_path, dest_path)

# # Copy images, uncomment below if you haven't copied them yet
# copy_images(train_df, 'Datasets/trainingset')
# copy_images(val_df, 'Datasets/validationset')

Training set shape: (17669, 4)
Validation set shape: (3119, 4)


In [7]:
# Augmented code to read the data, only uncomment if you dont have cell_types_df.pkl

# cell_types_df = pd.read_pickle("imagepaths.pkl")

# cell_types_df = cell_types_df.rename(columns={"Filepath":"Filepath", "Label":"Cell_Types"}) # To align with the names used for Deepheme
# cell_types_df['Cell_Types'] = cell_types_df['Cell_Types'].astype('category') # Converting type of columns to 'category'
# cell_types_df['Cell_Types_Cat'] = cell_types_df['Cell_Types'].cat.codes # Assigning a numerical value to each category and storing in another column

# # Create one hot encoding for each category
# enc = OneHotEncoder(handle_unknown='ignore') # passing bridge-types-cat column (label encoded values of bridge_types)
# enc_df = pd.DataFrame(enc.fit_transform(cell_types_df[['Cell_Types_Cat']]).toarray()) # merge with main df bridge_df on key values
# cell_types_df = cell_types_df.join(enc_df)

# #drop duplicates to create a reference list of the encoding for later use
# cell_types_df = cell_types_df.drop_duplicates(subset = 'Cell_Types', keep='first')

# # Store in pickle file for reuse
# cell_types_df.to_pickle("cell_types_df.pkl")
# cell_types_df.head()

In [8]:
cell_types_df = pd.read_pickle("cell_types_df.pkl")
cell_types_df.head()

,Filepath,Cell_Types,Final diagnosis,Classnames,Cell_Types_Cat,0,1,2
0,D:\Mathijs\Open Universiteit\Thesis\Implementa...,Other,AML,LY,2,0.0,0.0,1.0
1,D:\Mathijs\Open Universiteit\Thesis\Implementa...,Myeloid,AML,BL,1,0.0,1.0,0.0
35,D:\Mathijs\Open Universiteit\Thesis\Implementa...,Lymphoid,B-ALL,BL,0,1.0,0.0,0.0


In [36]:
# Load the data, split it into training and validation dataframes
df = pd.read_pickle('imagepaths.pkl')

train_df, val_df = train_test_split(df, test_size=0.15, random_state=42, stratify=df['Label'])
print(f"Training set shape: {train_df.shape}          Training set label count: {str(Counter(train_df['Label'].to_list()))[7:][1:][:-1]} \n")
print(f"Validation set shape: {val_df.shape}         Validation set label count: {str(Counter(val_df['Label'].to_list()))[7:][1:][:-1]} \n")

Training set shape: (17669, 4)          Training set label count: {'Other': 10069, 'Myeloid': 6582, 'Lymphoid': 1018} 

Validation set shape: (3119, 4)         Validation set label count: {'Other': 1777, 'Myeloid': 1162, 'Lymphoid': 180} 



In [37]:
## Simple normalization to improve the data generalibility

transform_pipeline = albumentations.Compose(
    [
        albumentations.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        
    ]
)

In [40]:
resnext50_pretrained = torch.hub.load('pytorch/vision:v0.10.0', 'resnext50_32x4d')
My_model = Myresnext50(my_pretrained_model= resnext50_pretrained, num_classes = 3)

Using cache found in C:\Users\moone/.cache\torch\hub\pytorch_vision_v0.10.0


In [38]:
# Run this cell if you want to utilize checkpoints, otherwise run cell above
resnext50_pretrained = torch.hub.load('pytorch/vision:v0.10.0', 'resnext50_32x4d')
My_model = Myresnext50(my_pretrained_model= resnext50_pretrained, num_classes = 3)

checkpoint_PATH = '../Result/ResNeXt50_pretrained/C_V/control/checkpoint_best_iteration3.ckpt'
checkpoint = torch.load(checkpoint_PATH)

# Function to remove 'module.' from the checkpoints, which must be removed because we do not use torch.nn.DataParallel
from collections import OrderedDict
def remove_data_parallel(old_state_dict):
    new_state_dict = OrderedDict()

    for k, v in old_state_dict.items():
        
        name = k[7:] # remove `module.`
        
        new_state_dict[name] = v
    
    return new_state_dict

checkpoint  = remove_data_parallel(checkpoint['model_state_dict'])

My_model.load_state_dict(checkpoint, strict=True)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to C:\Users\moone/.cache\torch\hub\v0.10.0.zip


FileNotFoundError: [Errno 2] No such file or directory: '../Result/ResNeXt50_pretrained/C_V/control/checkpoint_best_iteration3.ckpt'

In [5]:
#X_test_orig = cellnames[cellnames['dir'].isin(X_test)]
My_model = My_model.cuda().eval()
Orig_img = Img_DataLoader(img_list= X_val, split='viz',df= cell_types_df,transform = transform_pipeline)
shuffle = False
dataloader = DataLoader(Orig_img, batch_size=32, num_workers=2, shuffle=shuffle)


for i, _batch in enumerate(dataloader):
    
    if i == 0:

        images = _batch["image"].cuda()
        label = _batch["label"]
        ID    = [x.split('/')[-2]+"_"+x.split('/')[-1] for x in _batch['ID']]
        pred_prob = My_model(images)
        pred_hidden_layer = My_model.pretrained(images)

        pred_prob = torch.flatten(pred_prob, start_dim=1).detach().cpu().numpy()
        label = torch.flatten(label, start_dim=1).cpu().numpy()
        pred_hidden_layer = torch.flatten(pred_hidden_layer, start_dim=1).detach().cpu().numpy()
    else:
        images = _batch["image"].cuda()
        _label = _batch["label"]
        _ID    = [x.split('/')[-2]+"_"+x.split('/')[-1] for x in _batch['ID']]
        _pred_prob = My_model(images)
        _pred_hidden_layer = My_model.pretrained(images)
        
        _pred_prob = torch.flatten(_pred_prob, start_dim=1).detach().cpu().numpy()
        _label = torch.flatten(_label, start_dim=1).cpu().numpy()  
        _pred_hidden_layer = torch.flatten(_pred_hidden_layer, start_dim=1).detach().cpu().numpy()
        
        ID = ID + _ID
        pred_prob = np.concatenate((pred_prob, _pred_prob))
        label = np.concatenate((label, _label))
        pred_hidden_layer = np.concatenate((pred_hidden_layer, _pred_hidden_layer))
pred_prob = softmax(pred_prob, axis=1)
print(pred_prob.shape)
print(label.shape)
print(pred_hidden_layer.shape)
print(len(ID))
#pred_class_binary = np.where(res,np.max(res, axis =1))

for i in range(pred_prob.shape[0]):
    if i == 0:
        predict_class = np.where(pred_prob[i] == np.max(pred_prob[i]),1,0).reshape(1,23)
    else:
        _predict_class = np.where(pred_prob[i] == np.max(pred_prob[i]),1,0).reshape(1,23)
        predict_class = np.concatenate((predict_class, _predict_class), axis =0)
print(predict_class.shape)

/home/aa-ssun2-cmp/anaconda3/envs/Harry_Img/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


(5959, 23)
(5959, 23)
(5959, 1000)
5959
(5959, 23)


In [6]:
label = pd.DataFrame(data=label, index=ID, columns=cell_types_df['Cell_Types'].tolist())
pred_prob = pd.DataFrame(data=pred_prob, index=ID, columns=cell_types_df['Cell_Types'].tolist())
pred_class = pd.DataFrame(data=predict_class, index=ID, columns=cell_types_df['Cell_Types'].tolist())

In [7]:
from sklearn.metrics import (roc_auc_score, accuracy_score, recall_score, precision_score, f1_score)
from scipy.special import softmax
def evaluation_metrics_multiclass(label, pred_prob, pred_class):
    #creating a set of all the unique classes using the actual class list
    classes = label.columns
    eval_dict = {}
    
    for per_class in classes:
        #creating a list of all the classes except the current class 
        groundtruth_per_class  = label[per_class].tolist()
        pred_prob_per_class    = pred_prob[per_class].tolist()
        pred_class_per_class = pred_class[per_class].tolist()
        #marking the current class as 1 and all other classes as 0

        roc_auc = roc_auc_score(groundtruth_per_class, pred_prob_per_class)
        f1 = f1_score(groundtruth_per_class, pred_class_per_class)
        acc = accuracy_score(groundtruth_per_class, pred_class_per_class)
        precision = precision_score(groundtruth_per_class, pred_class_per_class)
        recall = recall_score(groundtruth_per_class, pred_class_per_class)
        used_metrics = ['AUC','F1','Acc','Precision','Recall']
        eval_dict[per_class] = [roc_auc, f1, acc, precision, recall]

    return eval_dict


# assuming your already have a list of actual_class and predicted_class from the logistic regression classifier
multiclass = evaluation_metrics_multiclass(label, pred_prob, pred_class)
print(multiclass)

{'B1': [0.9904544453123436, 0.87292817679558, 0.9961402919953012, 0.8229166666666666, 0.9294117647058824], 'B2': [0.9183891784561926, 0.7931034482758621, 0.9979862393018963, 0.9583333333333334, 0.6764705882352942], 'E0': [0.9910738900375731, 0.9379968203497616, 0.9934552777311629, 0.9516129032258065, 0.9247648902821317], 'E4': [0.9768431265262283, 0.75, 0.9939587179056889, 0.7012987012987013, 0.8059701492537313], 'ER1': [0.988114981684708, 0.7397260273972603, 0.9744923644906863, 0.717607973421927, 0.7632508833922261], 'ER2': [0.9810824798117065, 0.7717041800643087, 0.9761704984057727, 0.7619047619047619, 0.7817589576547231], 'ER3': [0.9866138823229731, 0.8422496570644717, 0.9807014599765062, 0.8872832369942196, 0.8015665796344648], 'ER4': [0.9979856184165578, 0.9234629861982434, 0.9897633831179729, 0.8846153846153846, 0.9658792650918635], 'ER5': [0.9982017912423314, 0.9520000000000001, 0.9959724786037926, 0.918918918918919, 0.9875518672199171], 'ER6': [0.9842606862966932, 0.94915254237

In [8]:
df = pd.DataFrame.from_dict(multiclass)
df.index = ['AUC','F1','Acc','Precision','Recall']
df.T

,AUC,F1,Acc,Precision,Recall
B1,0.990454,0.872928,0.996140,0.822917,0.929412
B2,0.918389,0.793103,0.997986,0.958333,0.676471
E0,0.991074,0.937997,0.993455,0.951613,0.924765
E4,0.976843,0.750000,0.993959,0.701299,0.805970
ER1,0.988115,0.739726,0.974492,0.717608,0.763251
ER2,0.981082,0.771704,0.976170,0.761905,0.781759
ER3,0.986614,0.842250,0.980701,0.887283,0.801567
ER4,0.997986,0.923463,0.989763,0.884615,0.965879
ER5,0.998202,0.952000,0.995972,0.918919,0.987552
ER6,0.984261,0.949153,0.995972,0.991150,0.910569


In [9]:
from utils.utils import one_vs_rest_metrics, get_overall_multiclass_metrics

In [10]:
df_one_vs_rest = one_vs_rest_metrics(np.argmax(label.to_numpy(), axis=1),
                    np.argmax(pred_class.to_numpy(), axis=1), 
                    pred_prob.to_numpy())
df_one_vs_rest.index = cell_types_df['Cell_Types'].tolist()
df_one_vs_rest

,auc,f1,n_true,n_pred
B1,0.990454,0.872928,85,96
B2,0.918389,0.793103,34,24
E0,0.991074,0.937997,319,310
E4,0.976843,0.750000,67,77
ER1,0.988115,0.739726,283,301
ER2,0.981082,0.771704,307,315
ER3,0.986614,0.842250,383,346
ER4,0.997986,0.923463,381,416
ER5,0.998202,0.952000,241,259
ER6,0.984261,0.949153,246,226


In [15]:

df_overall_multiclass_metrics = get_overall_multiclass_metrics(np.argmax(label.to_numpy(), axis=1),
                    np.argmax(pred_class.to_numpy(), axis=1), 
                    pred_prob.to_numpy())
df_overall_multiclass_metrics

{'accuracy': 0.8531632824299379,
 'balanced_accuracy': 0.8530460389601365,
 'auc_ovr_macro': 0.9865231308846351,
 'auc_ovr_weighted': 0.9880544259481447}